<a href="https://colab.research.google.com/github/kerryback/2022-638-binder/blob/main/9-targets-and-industries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymssql


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 1.6/1.6 MB 50.5 MB/s eta 0:00:00
     ---------------------------------------- 190.9/190.9 kB ? eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import QuantileTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from joblib import dump

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.compose import TransformedTargetRegressor

In [4]:
server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

### Transforming target

In [5]:
data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()

In [7]:
transform1 = QuantileTransformer(
    output_distribution="normal"
)

transform2 = QuantileTransformer(
    output_distribution="normal"
)

transform3 = QuantileTransformer(
    output_distribution="normal"
)

poly = PolynomialFeatures(degree=2)

model = TransformedTargetRegressor(
    regressor=RandomForestRegressor(random_state=0),
    transformer=transform3
)

pipe = make_pipeline(
  transform1,
  poly,
  transform2,
  model
)

param_grid = {
    "transformedtargetregressor__regressor__max_depth": [3, 4, 5]
}

cv = GridSearchCV(
  pipe, 
  param_grid=param_grid
)

X = data[["roeq", "mom12m"]]
y = data["ret"]

cv.fit(X, y)
cv.score(X, y)

0.10282019584138602

### Industries

In [8]:
data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m, siccd
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()

ff49 = pd.read_excel("files/ff49.xlsx", index_col=0)
def industry(sic):
  try:
    return ff49[(ff49.lower<=sic)&(sic<=ff49.upper)].index[0]
  except:
    return "Almost Nothing"
    
data["industry"] = data.siccd.map(industry)

data["roeqx"] = data.groupby("industry").roeq.transform(
    lambda x: x - x.mean()
)
data["mom12mx"] = data.groupby("industry").mom12m.transform(
    lambda x: x - x.mean()
)
data["mom12mi"] = data.groupby("industry").mom12m.transform("mean")

transform1 = QuantileTransformer(
    output_distribution="normal"
)

transform2 = QuantileTransformer(
    output_distribution="normal"
)

transform3 = QuantileTransformer(
    output_distribution="normal"
)

poly = PolynomialFeatures(degree=2)

model = TransformedTargetRegressor(
    regressor=RandomForestRegressor(random_state=0),
    transformer=transform3
)

### Without dummies

In [9]:

pipe = make_pipeline(
  transform1,
  poly,
  transform2,
  model
)

param_grid = {
    "transformedtargetregressor__regressor__max_depth": [3, 4, 5]
}

cv = GridSearchCV(
  pipe, 
  param_grid=param_grid
)

X = data[["roeq", "mom12m", "roeqx", "mom12mx", "mom12mi"]]
y = data["ret"]

cv.fit(X, y)


0.14457511396868217

### With dummies

In [ ]:
numeric_transform = make_pipeline(
  transform1,
  poly,
  transform2
)

category_transform = OneHotEncoder()

feature_transform = make_column_transformer(
  (numeric_transform, ["roeq", "mom12m", "roeqx", "mom12mx", "mom12mi"]),
  (category_transform, ["industry"])
)

pipe = make_pipeline(
  feature_transform,
  model
)

param_grid = {
    "transformedtargetregressor__regressor__max_depth": [3, 4, 5]
}

cv = GridSearchCV(
  pipe, 
  param_grid=param_grid
)

X = data[["roeq", "mom12m", "roeqx", "mom12mx", "mom12mi", "industry"]]
y = data["ret"]

cv.fit(X, y)